In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [ ]:
from keras.layers import Dense,RepeatVector
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize
from elasticsearch_dsl import Search

In [ ]:
from datetime import timedelta, datetime

In [ ]:
import time

In [ ]:
import pymysql

connection = pymysql.connect(host='mysql.cw34ebrphxxg.eu-central-1.rds.amazonaws.com',
                         user='admin',
                         password='jazeera123',
                         db='TEST_dB')

cursor=connection.cursor()

In [ ]:
elastic = Elasticsearch("https://es.jazeerapaints.com/")

In [ ]:

def unique_items(Region,Channel):
    
    list=[]
    df=elastic.search(index="jaz*", body={
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "region": Region
              }
            },
            {
              "match": {
                "saleschannel": Channel
              }
            }
          ]
        }
      },
                          "aggs" : {
                              "products" : {
                                  "terms" : {
                                      "field" : "itemid.keyword",
                                      "size" : 9000
                                   }
                              }
                          }
                          })
    
    for aggregation in df["aggregations"]["products"]["buckets"]:
        list.append(aggregation['key'])
        
    return list

In [ ]:
def unique_colours(Region,Channel,itemid):
    list=[]
    df=elastic.search(index="jaz*", body={
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "region": Region
              }
            },
            {
              "match": {
                "saleschannel": Channel
              }
            },
             {
              "match": {
                "itemid": itemid
              }
            } 
              
          ]
        }
      },
                          "aggs" : {
                              "colour" : {
                                  "terms" : {
                                      "field" : "colour.keyword",
                                      "size" : 9000
                                   }
                              }
                          }
                          })
    
    for aggregation in df["aggregations"]["colour"]["buckets"]:
        list.append(aggregation['key'])
        
    return list

In [ ]:
def unique_sizes(Region,Channel,itemid,colour):
    list=[]
    df=elastic.search(index="jaz*", body={
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "region": Region
              }
            },
            {
              "match": {
                "saleschannel": Channel
              }
            },
             {
              "match": {
                "itemid": itemid
              }
            },
              {
              "match": {
                "colour": colour
              }
            }
              
          ]
        }
      },
                          "aggs" : {
                              "size" : {
                                  "terms" : {
                                      "field" : "size.keyword",
                                      "size" : 9000
                                   }
                              }
                          }
                          })
    
    for aggregation in df["aggregations"]["size"]["buckets"]:
        list.append(aggregation['key'])
        
    return list

In [ ]:
def dimensions(Region,Channel,itemid,colour,size):
    
    df11 = elastic.search(index="jaz*", body={
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "region": Region
              }
            },
            {
              "match": {
                "saleschannel": Channel
              }
            },{
                  "match": {
                    "itemid": itemid
                  }
                },
                  {
                  "match": {
                    "colour": colour
                  }
                },
                  {
                  "match": {
                    "size": size
                  }
                }
          ]
        }
      },     "aggs": {
        "amount_per_day": {
          "date_histogram": {
            "field": "invoicedate",
            "interval": "day",
            "format" : "yyyy-MM-dd"
          },
          "aggs": {
            "total_amount": {
              "sum": {
                "field": "actualvalue"
              }
            }
          }
        }
      }

    },size=9000)
    
    list_dates=[]
    list_values=[]
    for aggregation in df11["aggregations"]["amount_per_day"]["buckets"]:

        list_dates.append(aggregation['key_as_string'])
        list_values.append(aggregation['total_amount']['value'])
    df1 = pd.DataFrame({'INVOICEDATE' : list_dates,'LINEAMOUNT' : list_values})
    
    return df1

In [ ]:
def Forecast1(Region,channel,Forecast_days):
    count_i=0
    ID=0
    elastic = Elasticsearch("https://es.jazeerapaints.com/")
    u_items = unique_items(Region,channel)
    check='HSN'
    Remove=[u_items.remove(x) for x in u_items if x.lower().startswith(check.lower())]
    count_items=len(u_items)
    doc1 = {'Total no. of items':count_items,
            'timestamp': datetime.now()}
    ID+=1
    res = elastic.index(index="log-index", id=ID, body=doc1)
    elastic.indices.refresh(index="log-index")
    print("Total no. of items-",count_items) 
    for i in u_items:
        count_i+=1    
        u_colors = unique_colours(Region,channel,i)
        count_colours=len(u_colors)
        doc2 = {
            'Name of the item currently running':i,
            'Total no. of colours present in item':count_colours,
            'timestamp': datetime.now()}
        ID+=1
        res = elastic.index(index="log-index", id=ID, body=doc2)
        elastic.indices.refresh(index="log-index")
        print("Total no. of colours present in item-",i,"is",count_colours)
        count_c=0        
        for k in u_colors:
            count_c+=1
            sizes = unique_sizes(Region,channel,i,k)
            count_sizes=len(sizes)
            doc3 = {
                'Name of the item currently running':i,
                'Name of the colour currently running':k,
                'Total no. of sizes present in color':count_sizes,
                'timestamp': datetime.now()}
            ID+=1
            res = elastic.index(index="log-index", id=ID, body=doc3)
            elastic.indices.refresh(index="log-index")
            print("Total no. of sizes present in item-",i,"colour-",k,"is",count_sizes)
            count_s=0            
            for j in sizes:
                count_s+=1
                temp=dimensions(Region,channel,i,k,j)
                temp['INVOICEDATE'] = pd.to_datetime(temp['INVOICEDATE'])
                temp['LINEAMOUNT'] = temp['LINEAMOUNT'].astype(float)
                last_date=temp['INVOICEDATE'].iloc[-1]
                Forecast_first_date = last_date + timedelta(1)
                try:                    
                    reg=np.full(shape=Forecast_days,fill_value=Region)
                    chnl=np.full(shape=Forecast_days,fill_value=channel)
                    product=np.full(shape=Forecast_days,fill_value=i)
                    col=np.full(shape=Forecast_days,fill_value=k)
                    dim=np.full(shape=Forecast_days,fill_value=j)                    
                    d = {'Region':reg,'SalesChannel':chnl,'ITEMID':product,'Colour':col,'SIZE':dim}
                    df_Forecasted_all = pd.DataFrame(d)
                    df_Forecasted_all['UNIQ_ID'] = df_Forecasted_all[['ITEMID','Colour','SIZE']].apply(lambda x: '_'.join(x), axis = 1) 
                    df_Forecasted_all['UNIQ_ID'] =df_Forecasted_all['Region'].astype(str)+'_'+df_Forecasted_all['SalesChannel'].astype(str)+'_'+df_Forecasted_all['UNIQ_ID']
                    combination=df_Forecasted_all['UNIQ_ID'].iloc[-1]                    
                    Train=temp
                    train = Train.iloc[:, 1:2].values                    
                    scaler = MinMaxScaler()
                    train_df = train.reshape(-1,1)
                    scaler.fit(train_df)
                    train_df = scaler.transform(train_df)                    
                    def create_dataset(dataset, look_back):
                        dataX, dataY = [], []
                        for i in range(len(dataset)-look_back):
                            a = dataset[i:(i + look_back), 0]
                            dataX.append(a)
                            dataY.append(dataset[i + look_back, 0])
                        return np.array(dataX), np.array(dataY)                    
                    time_steps = 90
                    X_train, y_train = create_dataset(train_df, time_steps)                    
                    X_train = np.reshape(X_train, (X_train.shape[0], time_steps, 1))                    
                    print("Model running for Item-",i,"colour-",k,"size-",j)                    
                    model = Sequential()
                    model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(90,1)))
                    model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
                    model.add(MaxPooling1D(pool_size=2))
                    model.add(Flatten())
                    model.add(RepeatVector(30))
                    model.add(LSTM(128, activation='relu'))
                    model.add(Dense(100, activation='relu'))
                    model.add(Dense(1))                    
                    model.compile(metrics = ['accuracy'], optimizer = 'adam', loss = 'mean_squared_error')                    
                    history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_split=.10)                    
                    pred_list = []
                    Last_set=train_df[-90:]
                    batch = np.reshape(Last_set, (1, 90, 1))
                    for z in range(Forecast_days):  
                        pred = model.predict(batch)
                        pred_list.append(pred[0,0]) 
                        pred = pred.reshape(1,1,1) 
                        batch = np.concatenate((batch[:,1:,:], pred), axis=1)
                    pred_list_array=np.array(pred_list)
                    pred_list_array=pred_list_array.reshape(-1, 1)
                    pred_list_Unscaled = scaler.inverse_transform(pred_list_array)              
                    df_Forecasted = pd.DataFrame(pred_list_Unscaled, columns=["FORECASTVALUE"])
                    forecast_dates=pd.date_range(Forecast_first_date, periods=Forecast_days)
                    df_Forecasted_dates = pd.DataFrame(forecast_dates, columns=['DATE'])
                    df_Forecasted_dates=df_Forecasted_dates.astype(str)                    
                    temp_df = pd.concat([df_Forecasted_all,df_Forecasted_dates,df_Forecasted], axis = 1)                   
                    cols = "`,`".join([str(i) for i in temp_df.columns.tolist()])
                    for l,row in temp_df.iterrows():
                        sql = "INSERT INTO `TESTFORECAST_OUTPUT` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
                        cursor.execute(sql, tuple(row))
                        connection.commit()
                    doc4 = {
                            'Name of the combination completed': combination,
                            'No. of items remaining': count_items-count_i,
                            'No. of colours remaining':count_colours-count_c,
                            'No. of sizes remaining': count_sizes-count_s,
                            'timestamp': datetime.now()
                      
                        }
                    ID=ID+1
                    res = elastic.index(index="log-index", id=ID, body=doc4)
                    res = elastic.get(index="log-index", id=ID)
                    print(res['_source'])
                    elastic.indices.refresh(index="log-index")
                    print("Name of the combination completed",df_Forecasted_all['UNIQ_ID'].iloc[-1])
                    print("No. of items remaining",count_items-count_i)
                    print("No.of colours remaining in item-",i,"is",count_colours-count_c)
                    print("No. of sizes remaining in item-",i,"colour-",k,"is",count_sizes-count_s)                 
                except Exception as e:
                    Error=str(e)
                    print(Error)
                    localtime = time.asctime( time.localtime(time.time()))
                    combination=str(combination)
                    localtime=str(localtime)
                    sql = "INSERT INTO TESTERROR_LOGS (UNIQ_ID,ERROR_DATE,ERROR) VALUES (%s, %s, %s)"
                    val = (combination,localtime,Error)
                    cursor.execute(sql, val)
                    connection.commit()
                    print("Exception handled")

In [ ]:
x=Forecast1(4,3,90)